## Currents Analysis
This notebook processes and visualizes Salish Sea NEMO currents. Results are obtained from the Nowcast system through the ERDDAP server using the `xarray` library.

In [9]:
import numpy as np
import xarray as xr
import datetime as dtm
import dateutil.parser as dparser
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import myvisualizations as myviz
import os

from salishsea_tools import viz_tools, tidetools

#%matplotlib inline
#mpl.rcParams.update({'font.size': 12})
#mpl.rcParams["axes.formatter.useoffset"] = False

In [15]:
timerange = ['2016-04-01 00:00', '2016-04-02 00:00']
animate_currents_winds(timerange)

/ocean/bmoorema/research/MEOPAR/analysis-ben/visualization/NEMOcurrents_0m_2016Apr01T00to2016Apr02T00.mp4


In [14]:
def animate_currents_winds(timerange, depth=0, spacing={'NEMO': 5, 'GEM': 5},
                           window={'x_NEMO': slice(100, 398), 'x_GEM': slice(300000, 425000),
                                   'y_NEMO': slice(230, 570), 'y_GEM': slice(237500, 412500)},
                           writepath='/ocean/bmoorema/research/MEOPAR/analysis-ben/visualization'):
    """
    """
    
    # Parse timerange
    starttime, endtime = map(dparser.parse, timerange)
    frames = (endtime - starttime).days * 24
    winlen = dtm.timedelta(hours=20)
    timeslice = slice(starttime - winlen, endtime + winlen)
    
    # Write file path
    writefile = 'NEMOcurrents_{}m_{}to{}.mp4'.format(
                depth, starttime.strftime('%Y%b%dT%H'), endtime.strftime('%Y%b%dT%H'))
    filepath = os.path.join(writepath, writefile)
    
    # Load Nowcast and GEM results from ERDDAP using xarray
    GEM_OP    = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
    GEM_grid  = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaAtmosphereGridV1')
    u_vel     = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn3DuVelocity1hV1')
    v_vel     = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn3DvVelocity1hV1')
    NEMO_grid = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetry2V1')
    
    # Slice results objects
    lon_NEMO = NEMO_grid.longitude.sel(
        gridX=window['x_NEMO'], gridY=window['y_NEMO'])[1::spacing['NEMO'], 1::spacing['NEMO']]
    lat_NEMO = NEMO_grid.latitude.sel(
        gridX=window['x_NEMO'], gridY=window['y_NEMO'])[1::spacing['NEMO'], 1::spacing['NEMO']]
    lon_GEM = GEM_grid.longitude.sel(
        gridX=window['x_GEM'], gridY=window['y_GEM'])[::spacing['GEM'], ::spacing['GEM']]-360
    lat_GEM = GEM_grid.latitude.sel(
        gridX=window['x_GEM'], gridY=window['y_GEM'])[::spacing['GEM'], ::spacing['GEM']]
    u_NEMO = u_vel.uVelocity.sel(
        time=timeslice, gridX=window['x_NEMO'], gridY=window['y_NEMO']).sel(depth=depth, method='nearest')
    v_NEMO = v_vel.vVelocity.sel(
        time=timeslice, gridX=window['x_NEMO'], gridY=window['y_NEMO']).sel(depth=depth, method='nearest')
    u_GEM = GEM_OP.u_wind.sel(
        time=timeslice, gridX=window['x_GEM'], gridY=window['y_GEM'])[:, ::spacing['GEM'], ::spacing['GEM']]
    v_GEM = GEM_OP.v_wind.sel(
        time=timeslice, gridX=window['x_GEM'], gridY=window['y_GEM'])[:, ::spacing['GEM'], ::spacing['GEM']]
    
    # Unstagger NEMO currents
    u_NEMO, v_NEMO = viz_tools.unstagger(u_NEMO, v_NEMO)
    v_NEMO = v_NEMO.reindex_like(u_NEMO)
    u_NEMO, v_NEMO = viz_tools.rotate_vel(u_NEMO[:, ::spacing['NEMO'], ::spacing['NEMO']],
                                          v_NEMO[:, ::spacing['NEMO'], ::spacing['NEMO']], origin='grid')
    
    # Apply Doodson filter to currents and wind
    u_NEMO_filtered = tidetools.filter_timeseries(u_NEMO, method='doodson')
    v_NEMO_filtered = tidetools.filter_timeseries(v_NEMO, method='doodson')
    u_GEM_filtered  = tidetools.filter_timeseries(u_GEM,  method='doodson')
    v_GEM_filtered  = tidetools.filter_timeseries(v_GEM,  method='doodson')
    
    # Organize results into dictionaries
    coords     = {'lon_NEMO': lon_NEMO,        'lon_GEM': lon_GEM,
                  'lat_NEMO': lat_NEMO,        'lat_GEM': lat_GEM}
    unfiltered = {  'u_NEMO':   u_NEMO,          'u_GEM':   u_GEM,
                    'v_NEMO':   v_NEMO,          'v_GEM':   v_GEM}
    filtered   = {  'u_NEMO':   u_NEMO_filtered, 'u_GEM':   u_GEM_filtered,
                    'v_NEMO':   v_NEMO_filtered, 'v_GEM':   v_GEM_filtered}
    
    # Generate figure panels
    fig, ax = plt.subplots(1, 2, figsize=(15, 10))
    
    # Plot initial vector fields
    plot_objs_unfiltered = myviz.plot_currents(ax[0], starttime, coords, unfiltered)
    plot_objs_filtered   = myviz.plot_currents(ax[1], starttime, coords, filtered)
    
    # Next frame definition
    def update_plot(t, plot_objs_unfiltered, plot_objs_filtered):
        time = (starttime + dtm.timedelta(hours=t)).strftime('%a %Y-%m-%d %H:%M:%S')
        plot_objs_unfiltered = myviz.update_currents(time, plot_objs_unfiltered, unfiltered)
        plot_objs_filtered   = myviz.update_currents(time, plot_objs_filtered,   filtered)
        return plot_objs_unfiltered, plot_objs_filtered
    
    # Animate
    mywriter = animation.FFMpegWriter(fps=12, bitrate=600)
    ani = animation.FuncAnimation(fig, update_plot, fargs=(plot_objs_unfiltered, plot_objs_filtered),
                                  frames=frames, blit=False)
    ani.save(filepath, writer=mywriter)